In [1]:
import os
import h5py

In [2]:
root_dir = os.getcwd()
pre_t_filepath = os.path.join(root_dir, "mask_rcnn_prostate_pretrained.h5")
resnet_filepath = os.path.join(root_dir, "logs/prostate20180418T1255-held-out-set3(full)/mask_rcnn_prostate_0040.h5")

In [3]:
def combine_tumor_classifier(pre_t_filepath, resnet_filepath):
    """This function combine the pretrained maskrcnn weights with the tumor classifier weights.
    Function copy the tumor head from resnet file to pretrained file and save in the pretrained
    file.
    # Arguments
    pre_t_filepath: specify the file location of the pretrained maskrcnn
    resnet_filepath: specify the file location of the trained tumor classifier head
    """
    
    # read in the file
    pre_t_f = h5py.File(pre_t_filepath, mode='r+')
    resnet_f = h5py.File(resnet_filepath, mode='r')
    
    # delete all the tumor weights in the pre_t_f
    for name in pre_t_f:
        if name.startswith('tumor'):
            del pre_t_f[name]
    
    # retrieve the object attributes 'layer_names'
    layer_names = [n.decode('utf8') for n in pre_t_f.attrs['layer_names']]
    # delete all the 'layer_names' starts with 'tumor'
    for name in layer_names:
        if name.startswith('tumor'):
            layer_names.remove(name)
    
    # copy the tumor head from resnet_f to pre_t_f
    for name in resnet_f:
        if name.startswith('tumor'):
            resnet_f.copy(name, pre_t_f)
            layer_names.append(name)
    
    # modify the attribute 'layer_names'
    layer_names = [n.encode('utf8') for n in layer_names]
    pre_t_f.attrs.__delitem__('layer_names')
    pre_t_f.attrs.create('layer_names', layer_names)
    
    pre_t_f.close()
    resnet_f.close()

In [4]:
combine_tumor_classifier(pre_t_filepath, resnet_filepath)

In [5]:
# method to fetch the numpy data array from the h5 dataset object
# weight_pre = kenel_pre[()]